## using Adaboost classifier we will prepare a model on fraud data treating those who have taxable_income <= 30000 as "Risky" and others are "Good"

In [ ]:
#libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn import metrics

#Parameter tuning
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

#adaboost library
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
df = pd.read_csv("G:\Github\DS-assignments-python\Adaboost algorithm\Fraud_check.csv")

In [ ]:
df.head(10)

In [ ]:
df.info()

In [ ]:
df.nunique()

In [ ]:
df1 = df.copy()

In [ ]:
df1["Taxable.Income"] = pd.Series(np.where(df["Taxable.Income"]<= 30000 ,1,0))

In [ ]:
df1.rename(columns = {"Taxable.Income":"Tax_Inc_Risky"}, inplace = True)

Treating 1 as Risky and 0 as Good

In [ ]:
df1.head(10)

In [ ]:
df1["Tax_Inc_Risky"].value_counts()

In [ ]:
124/600

In [ ]:
#Creating dataframes with all numerical and dummies
dfd = pd.get_dummies(data = df1 , columns = ['Undergrad','Marital.Status','Urban'], drop_first = True)

In [ ]:
dfd.head()

#### so the data is nicely unbalance..
now adaboost is a algorithm which counters the imbalanced data  by giving more weight to datapoints which are not classified correctly... in this case minority class datapoints
Lets see how it performs

In [ ]:
dfd_x = dfd.iloc[:,1:]
dfd_y = dfd.iloc[:,0]

In [ ]:
dfd_tr_x,dfd_ts_x,dfd_tr_y,dfd_ts_y = train_test_split(dfd_x,dfd_y, test_size = 0.25,shuffle = True , random_state = 13)

AdaBoostClassifier(base_estimator=None, *, n_estimators=50, learning_rate=1.0, algorithm='SAMME.R', random_state=None)

- base_estimator - model that u wnat to have as classifier , default is decision tree
- n_estimator - number of stumps/tress / classifiaction learners you want to build
- learning rate - Weight applied to each classifier at each boosting iteration. A higher learning rate increases the contribution of each classifier. There is a trade-off between the learning_rate and n_estimators parameters.


In [ ]:
#Creating a adaboost model
mark1 = AdaBoostClassifier(n_estimators = 100)

In [ ]:
mark1.fit(dfd_tr_x,dfd_tr_y)

In [ ]:
pred_tr = mark1.predict(dfd_tr_x)
pred_ts = mark1.predict(dfd_ts_x)

In [ ]:
np.mean(pred_tr == dfd_tr_y)

In [ ]:
np.mean(pred_ts == dfd_ts_y)

In [ ]:
pd.crosstab(dfd_tr_y,pred_tr )

In [ ]:
pd.crosstab(dfd_ts_y,pred_ts)

The minority accuracy sucks even for adaboost.....why...lets try final with staified kfold sampling

In [ ]:
markx = AdaBoostClassifier()

In [ ]:
cv = StratifiedKFold(n_splits = 10,shuffle=True, random_state = 13)

In [ ]:
scores = cross_val_score(markx,dfd_x,dfd_y, scoring = "accuracy" , cv = cv)

In [ ]:
print("\n Accuracy is ", np.mean(scores),np.std(scores))

In [ ]:
scores

even after mainitaining the propartion of minority - majority the accuracy has not increased


Lets try oversampling the minority class
here the ammout is data is less ..so undersampling the data will result in loss of info and also considerably harm prediction models


### oversampling minority class


In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
os = SMOTE(sampling_strategy = 0.75)

In [ ]:
dfd_x1,dfd_y1 = os.fit_resample(dfd_x,dfd_y)

In [ ]:
print(dfd_x.shape,dfd_x1.shape)

In [ ]:
dfd_tr_x1,dfd_ts_x1,dfd_tr_y1,dfd_ts_y1 = train_test_split(dfd_x1,dfd_y1, test_size = 0.25,shuffle = True , random_state = 13)

In [ ]:
print(dfd_y.value_counts(),"\n",dfd_y1.value_counts())

In [ ]:
#Creating a adaboost model
mark4 = AdaBoostClassifier(n_estimators = 100)

In [ ]:
mark4.fit(dfd_tr_x1,dfd_tr_y1)

In [ ]:
pred_tr1 = mark4.predict(dfd_tr_x1)
pred_ts1 = mark4.predict(dfd_ts_x1)

In [ ]:
np.mean(pred_tr1 == dfd_tr_y1)

In [ ]:
np.mean(pred_ts1 == dfd_ts_y1)

In [ ]:
pd.crosstab(dfd_tr_y1,pred_tr1 )

In [ ]:
precision,recall,fscore,supp = precision_recall_fscore_support(dfd_tr_y1,pred_tr1, average = "binary")

print("Precision is -> %.2f" % precision)
print("recall is -> %.2f" % recall)
print("fscore is -> %.2f" % fscore)
print("support is -> ", supp)

In [ ]:
pd.crosstab(dfd_ts_y1,pred_ts1)

In [ ]:
#metrics.fbeta_score(dfd_ts_y1,pred_ts1,average = "binary",pos_label = )

In [ ]:
precision,recall,fscore,supp = precision_recall_fscore_support(dfd_ts_y1,pred_ts1, average = "binary")

In [ ]:
print("Precision is -> %.2f" % precision)
print("recall is -> %.2f" % recall)
print("fscore is -> %.2f" % fscore)
print("support is -> ", supp)

Startified sampling doesn't help and oversampling minority with smote doesn't help....the data is too bad..
600 bservation with 200 one type too less....i think need to try on larger dataset..to see if things changes